In [1]:
import os

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [3]:
load_dotenv()

True

In [4]:
key=os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbo", temperature=0.7)

d:\Projects to practice\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [7]:
with open("D:\Projects to practice\MCQGenerator\Response.json","r") as f:
    RESPONSE_JSON = json.load(f)

In [8]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
    
)

In [9]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    
)

In [12]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"], output_variables=["quiz", "review"], verbose=True)

In [14]:
PATH="D:\Projects to practice\MCQGenerator\data.txt"

In [15]:
with open(PATH, "r") as file:
    TEXT=file.read()

In [16]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [17]:
NUMBER=5
SUBJECT="AI"
TONE="Simple",
RESPONSE_JSON=RESPONSE_JSON

In [18]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain({
        "text":TEXT, 
        "number":NUMBER, 
        "subject":SUBJECT, 
        "tone":TONE, 
        "RESPONSE_JSON":json.dumps(RESPONSE_JSON)
        }
    )

d:\Projects to practice\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large 

In [19]:
print(response)

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:\n\nSystems, architecture and infrastructure for Generative AI\nMachine learning and Modeling using LLMs and Diffusion models\nLarge Language Models and its applications\nMulti-modal Generative AI and its applications\nGen AI b

In [20]:
quiz = response.get("quiz")

In [21]:
quiz_dict = json.loads(quiz)

In [22]:
print(quiz_dict)

{'1': {'no': '1', 'mcq': 'What is the focus of the workshop on Generative AI at AIMLSystems?', 'options': {'a': 'Building robots', 'b': 'Developing mobile apps', 'c': 'Building and deploying Generative AI systems', 'd': 'Creating virtual reality games'}, 'correct': 'c'}, '2': {'no': '2', 'mcq': 'Which of the following is NOT a topic of interest for the workshop?', 'options': {'a': 'Machine learning and Modeling using LLMs and Diffusion models', 'b': 'Evaluation of Language and Diffusion based models', 'c': 'Building self-driving cars', 'd': 'Responsible use of Gen AI'}, 'correct': 'c'}, '3': {'no': '3', 'mcq': 'What type of models are used in Generative AI according to the text?', 'options': {'a': 'Convolutional Neural Networks', 'b': 'Large Language Models', 'c': 'Recurrent Neural Networks', 'd': 'Decision Trees'}, 'correct': 'b'}, '4': {'no': '4', 'mcq': 'What is the focus of Multi-modal Generative AI according to the text?', 'options': {'a': 'Creating music', 'b': 'Generating text o

In [23]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}-> {option_value}" for option, option_value in value['options'].items()
        ]
    )
    correct = value['correct']
    quiz_table_data.append({'MCQ':mcq, "Choices":options, "Correct":correct})

In [24]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Building robots | b: Developing mobile apps | c: Building and deploying Generative AI systems | d: Creating virtual reality games',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is NOT a topic of interest for the workshop?',
  'Choices': 'a: Machine learning and Modeling using LLMs and Diffusion models | b: Evaluation of Language and Diffusion based models | c: Building self-driving cars | d: Responsible use of Gen AI',
  'Correct': 'c'},
 {'MCQ': 'What type of models are used in Generative AI according to the text?',
  'Choices': 'a: Convolutional Neural Networks | b: Large Language Models | c: Recurrent Neural Networks | d: Decision Trees',
  'Correct': 'b'},
 {'MCQ': 'What is the focus of Multi-modal Generative AI according to the text?',
  'Choices': 'a: Creating music | b: Generating text only | c: Generating content in multiple modalities | d: Building websites',
  'Correct': 'c'

In [25]:
df = pd.DataFrame(quiz_table_data)

In [26]:
df.to_csv("genai.xlxs")

In [27]:
import os
import logging
from datetime import datetime


LOG_FILE="{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [35]:
print(LOG_FILE)

{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log


In [36]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_12_2024_10_08_12'